<a href="https://www.kaggle.com/code/rubanzasilva/xgb-lgb-cat-neural-nets-with-fast-ai?scriptVersionId=199372882" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Loan Approval Prediction

The goal for this competition is to predict whether an applicant is approved for a loan.

# Library and Data import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv


In [2]:
%%time
%pip install catboost
%pip install optuna
#%pip install optuna_distributed
#%pip install openfe
%pip install seaborn
%pip install xgboost
%pip install lightgbm
%pip install fastkaggle
#%pip install h2o
%pip install -Uqq fastbook
#%pip install polars
#%pip install -q -U autogluon.tabular
#%pip install autogluon
#%pip install --upgrade pip
%pip install tqdm
#%pip install wandb
#%pip install sweetviz

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
CPU times: user 1.68 s, sys: 373 ms, total: 2.05 s
Wall time: 1min 34s


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm

import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)


from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier,StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score,train_test_split


from pathlib import Path
import os

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv


import warnings


#from openfe import OpenFE, transform
#from autogluon.tabular import TabularDataset, TabularPredictor

#import h2o
#from h2o.automl import H2OAutoML

import gc

import optuna
from optuna.samplers import TPESampler

import pickle
from joblib import dump, load
#import sweetviz as sv
#from IPython.display import FileLink

In [4]:
!ls /kaggle/input/playground-series-s4e10

sample_submission.csv  test.csv  train.csv


In [5]:
path = Path('/kaggle/input/playground-series-s4e10/')
path

Path('/kaggle/input/playground-series-s4e10')

In [6]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')

In [7]:
#comp = 'playground-series-s4e10'

#path = setup_comp(comp, install='fastai "timm>=0.6.2.dev0"')

In [8]:
#iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

#if iskaggle: path = Path('playground-series-s4e10/')
#else:
    #import zipfile,kaggle
    #path = Path('playground-series-s4e10')
    #kaggle.api.competition_download_cli(str(path))
    #zipfile.ZipFile(f'{path}.zip').extractall(pat

# Exploratory Data Analysis

In [9]:
train_df.shape

(58645, 12)

In [10]:
train_df.columns

Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file',
       'cb_person_cred_hist_length', 'loan_status'],
      dtype='object')

In [11]:
train_df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
id,,,,,,,,,,,,
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


# fast.ai preprocessing

In [12]:
cont_names,cat_names = cont_cat_split(train_df, dep_var='loan_status')
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))
to = TabularPandas(train_df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='loan_status',
                   y_block=CategoryBlock(),
                   splits=splits)
dls = to.dataloaders(bs=64)
#dls = to.dataloaders(bs=1024)
test_dl = dls.test_dl(test_df)
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

# Modelling

## Neural Network

In [13]:
learn = tabular_learner(dls, metrics=RocAucBinary())

In [14]:
learn.fit_one_cycle(10,1e-6)

epoch,train_loss,valid_loss,roc_auc_score,time
0,0.704793,0.711803,0.374813,00:07
1,0.661297,0.663798,0.545874,00:07
2,0.618422,0.616244,0.753658,00:07
3,0.593624,0.597416,0.832108,00:07
4,0.582465,0.573390,0.856398,00:07
5,0.580397,0.594011,0.866787,00:07
6,0.565400,0.574355,0.866165,00:07
7,0.570074,0.575217,0.867954,00:07
8,0.575389,0.561394,0.870122,00:07
9,0.571479,0.573547,0.870033,00:07


In [15]:
dl = test_dl

In [16]:
learn.save('nn_model_loan_approval_v1')

Path('models/nn_model_loan_approval_v1.pth')

In [17]:
#learn.load('nn_model_loan_approval_v1')  

In [18]:
test_df.shape

(39098, 11)

In [19]:
nn_preds_x1 = learn.get_preds(dl=dl)[0]

In [20]:
nn_preds_x1

tensor([[0.1947, 0.8053],
        [0.5886, 0.4114],
        [0.3239, 0.6761],
        ...,
        [0.2129, 0.7871],
        [0.0930, 0.9070],
        [0.0682, 0.9318]])

In [21]:
#i select the 2nd column of the nn_preds
final_preds = nn_preds_x1[:, 1]
final_preds

tensor([0.8053, 0.4114, 0.6761,  ..., 0.7871, 0.9070, 0.9318])

In [22]:
#sub_df['loan_status'] = final_preds
#sub_df.to_csv('submission.csv', index=False)
#sub = pd.read_csv('submission.csv')
#sub

In [23]:
#dl = test_dl
#nn_preds = learn.get_preds(dl=dl)
#nn_preds_x = learn.get_preds()[0]
#a_preds, _ = learn.get_preds(dl=dl)
#nn_preds_y = a_preds.squeeze(1)
#nn_preds_proba = (a_preds[:, 1])

## XGBoost

In [24]:
xgb_params = {
              
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'booster': 'gbtree',
    'n_estimators': 5000,
    'max_depth': 10,
    'min_child_weight': 17,  
    'colsample_bytree': 0.65, 
    'eta': 0.55,  
    'device' : 'cuda',
    #'device' : 'cuda:1',
    #subsample
    #gamma
    
    #lamda
    #alpha
    #'tree_method': 'hist' set to auto for default
    #seed
    #'max_bin': 52000, - only used if tree_method is set to hist    
    
    #'max_delta_step': 2, - usually not needed helps in class imbalance in regression
    
    #'tree_method': 'hist' - gpu_hist
    'tree_method':'auto',
    #'predictor': 'gpu_predictor',
    #'n_gpus':-1,
}

xgb_optuna_params = {
    'eta': 0.044303752452182654,
    'n_estimators': 7238,
    'colsample_bytree': 0.7616580256429863,
    'subsample': 0.6586623872476727,
    'max_depth': 7,
    'min_child_weight': 17, 
    'booster': 'gbtree',
    'tree_method':'hist',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'device':'cuda',
}

### Optuna optimization

In [25]:
#def objective(trial):
    #params = {
        #'num_leaves': trial.suggest_int('num_leaves', 100, 500),
        #'eta': trial.suggest_float('eta', 0.001, 1.0, log=True),
        #'n_estimators': trial.suggest_int('n_estimators', 300, 10000),
        #'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        #'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        #'max_depth': trial.suggest_int('max_depth', 1, 15),
        #'objective': 'binary:logistic',
        #'eval_metric': 'auc',
        #'device':'cuda',

    #}
    
    #K_FOLDS = 10  # Number of folds for cross-validation
    #kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

    #xgb_fold_scores = []
    
    #for train_index, test_index in kfold.split(train_df):
    #print(f'train: {len(train_index)} samples, test: {len(test_index)} samples')

    #for train_index, val_index in kfold.split(X_train):
        #X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
        #y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Train the model on the current fold
        
        #xgb_model_fold = xgb.XGBClassifier(**xgb_optuna_params)
        #xgb_model_fold.fit(X_fold_train, y_fold_train)
        
        # Predict probabilities on the validation set for the current fold
        #y_pred_fold = xgb_model_fold.predict_proba(X_fold_val)[:, 1]
        #y_pred_fold_tt = xgb_model_fold.predict(test_dl.xs)
        
        # Calculate and store the AUC-ROC score for the current fold
        #xgb_cv_score = roc_auc_score(y_fold_val, y_pred_fold)
        #xgb_fold_scores.append(xgb_cv_score)
        
        # Calculate the average score across all folds
        #return np.mean(xgb_fold_scores)

# Create and run the study
#study = optuna.create_study(sampler=TPESampler(n_startup_trials=30, multivariate=True, seed=0), direction="maximize")
#study.optimize(objective, n_trials=100)

#print('Best value:', study.best_value)
#print('Best trial:', study.best_trial.params)

### XGB model cross validation

In [26]:
K_FOLDS = 10  # Number of folds for cross-validation
kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
xgb_fold_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(X_train), 1):
    X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
    
    # Train the model on the current fold
    xgb_model_fold = xgb.XGBClassifier(**xgb_optuna_params)
    xgb_model_fold.fit(X_fold_train, y_fold_train)
    
    # Predict on the validation set for the current fold
    y_pred_fold = xgb_model_fold.predict_proba(X_fold_val)[:, 1]
    #y_pred_fold_tt = xgb_model_fold.predict(test_dl.xs)
    
    # Calculate and store the Matthews Correlation Coefficient for the current fold
    xgb_cv_score = roc_auc_score(y_fold_val, y_pred_fold)
    xgb_fold_scores.append(xgb_cv_score)
    
    print(f"Fold {fold} AUC: {xgb_cv_score}")

# Calculate and print the mean score across all folds
xgb_mean_score = np.mean(xgb_fold_scores)
print(f"Mean AUC: {xgb_mean_score}")

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:13:41] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


Fold 1 AUC: 0.9457057125681739


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:14:15] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


Fold 2 AUC: 0.9444423854655557


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:14:49] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


...

Fold 8 AUC: 0.9412765547658944


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:18:15] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


Fold 9 AUC: 0.9393546425843954


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:18:50] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


Fold 10 AUC: 0.9381048354847482
Mean AUC: 0.9464211419597948


### On full dataset

In [27]:
%%time
xgb_model = xgb.XGBClassifier(**xgb_optuna_params)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))
xgb_preds_proba = tensor(xgb_model.predict_proba(test_dl.xs))[:, 1]

xgb_preds_x = tensor(xgb_model.predict(X_test))
xgb_preds_x_proba = tensor(xgb_model.predict_proba(X_test))[:, 1]

xgb_score = roc_auc_score(y_test,xgb_preds_x_proba)
xgb_score

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:19:24] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


CPU times: user 1min 41s, sys: 530 ms, total: 1min 42s
Wall time: 51.4 s


0.9438791741500856

In [28]:
#submission
!rm submission.csv
#sub_df['loan_status'] = xgb_preds_proba
#sub_df.to_csv('submission.csv', index=False)
#sub = pd.read_csv('submission.csv')
#sub

rm: cannot remove 'submission.csv': No such file or directory


## LGBM

In [29]:
#%%time
#K_FOLDS = 10  # Number of folds for cross-validation
#kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
#lgbm_fold_scores = []

#for fold, (train_index, val_index) in enumerate(kf.split(X_train), 1):
    #X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
    #y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
    
    # Train the model on the current fold
    #lgbm_model_fold = lgb.LGBMClassifier(**lgbm_params)
    #lgbm_model_fold.fit(X_fold_train, y_fold_train)
    
    # Predict on the validation set for the current fold
    #y_pred_fold = lgbm_model_fold.predict_proba(X_fold_val)[:, 1]
    #y_pred_fold_tt = lgbm_model_fold.predict(test_dl.xs)
    
    # Calculate and store the Root Mean Squared Error for the current fold
    #lgbm_cv_score = roc_auc_score(y_fold_val, y_pred_fold)
    #lgbm_fold_scores.append(lgbm_cv_score)
    
    #print(f"Fold {fold} AUC: {lgbm_cv_score}")

# Calculate and print the mean score across all folds
#lgbm_mean_score = np.mean(lgbm_fold_scores)
#print(f"Mean RMSE: {lgbm_mean_score}")

## Random Forests

In [30]:
%%time
rf = RandomForestClassifier(100, min_samples_leaf=3)
rf_model = rf.fit(X_train, y_train);

rf_preds = tensor(rf_model.predict(test_dl.xs))
rf_preds_proba = tensor(rf_model.predict_proba(test_dl.xs))[:, 1]

rf_preds_x = tensor(rf_model.predict(X_test))
rf_preds_proba_x = tensor(rf_model.predict_proba(X_test))[:, 1]

rf_score = roc_auc_score(y_test,rf_preds_proba_x)
rf_score

CPU times: user 9.29 s, sys: 28.9 ms, total: 9.32 s
Wall time: 9.32 s


0.938886253440963

In [31]:
#!rm submission.csv
#sub_df['loan_status'] = rf_preds_proba
#sub_df.to_csv('submission.csv', index=False)
#sub = pd.read_csv('submission.csv')
#sub

## CatBoost

In [32]:
cat_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    #'class_names': [0, 1],
    'learning_rate': 0.75,
    'iterations': 5000,
    'depth': 9,
    'random_strength': 0,
    'l2_leaf_reg': 0.5,
    'max_leaves': 512,
    'fold_permutation_block': 64,
    #'task_type': 'GPU',
    #'devices': '0:1',
    'random_seed': 42,
    'verbose': False,
    'allow_writing_files': False
}

In [33]:
%%time
#using full ds
cat_model = CatBoostClassifier(**cat_params)
cat_model = cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

#test set preds
#cat_preds = tensor(cat_model.predict(test_dl.xs))
cat_preds_proba = tensor(cat_model.predict_proba(test_dl.xs))[:, 1]
#cat_preds_final = cat_preds.squeeze(1)

#validation set preds
#cat_preds_x = tensor(cat_model.predict(X_test))
cat_preds_proba_x = tensor(cat_model.predict_proba(X_test))[:, 1]

#cat_preds_x_final = cat_preds_x.squeeze(1)

#accuracy_score(y_test,cat_preds_x)

cat_score = roc_auc_score(y_test,cat_preds_proba_x)
cat_score

CPU times: user 4min 2s, sys: 21.6 s, total: 4min 23s
Wall time: 1min 12s


0.9423282443734559

In [34]:
#submission
!rm submission.csv
sub_df['loan_status'] = cat_preds_proba
sub_df.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

rm: cannot remove 'submission.csv': No such file or directory


,id,loan_status
0,58645,0.999822
1,58646,0.024066
2,58647,0.365321
3,58648,0.007175
4,58649,0.095743
...,...,...
39093,97738,0.062774
39094,97739,0.010239
39095,97740,0.011872
39096,97741,0.099137


# Ranking performance

In [35]:
AUC = pd.DataFrame({
    'algorithm': [ 'XGBoost','XGBoost CV','Random Forest',
                 ],
    'AUC': [xgb_score,xgb_mean_score,
            rf_score
           ]
})

AUC_sorted = AUC.sort_values(by='AUC', ascending=True)
AUC_sorted

,algorithm,AUC
2,Random Forest,0.938886
0,XGBoost,0.943879
1,XGBoost CV,0.946421
